## Imports & Inputs

In [1]:
from pathlib import Path
from typing import Callable, Iterable, List, Sequence

import git
import numpy as np
import numpy.typing as npt


REPO_ROOT = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PUZZLE_INPUT = REPO_ROOT / 'puzzle_inputs' / 'Day 4.txt'

assert PUZZLE_INPUT.exists()

<h2>--- Day 4: Giant Squid ---</h2><p>You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you <em>can</em> see, however, is a giant squid that has attached itself to the outside of your submarine.</p>
<p>Maybe it wants to play <a href="https://en.wikipedia.org/wiki/Bingo_(American_version)" target="_blank">bingo</a>?</p>
<p>Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is <em>marked</em> on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board <em>wins</em>. (Diagonals don't count.)</p>
<p>The submarine has a <em>bingo subsystem</em> to help passengers (currently, you and the giant squid) pass the time. It automatically generates a random order in which to draw numbers and a random set of boards (your puzzle input). For example:</p>
<pre><code>7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
</code></pre>
<p>After the first five numbers are drawn (<code>7</code>, <code>4</code>, <code>9</code>, <code>5</code>, and <code>11</code>), there are no winners, but the boards are marked as follows (shown here adjacent to each other to save space):</p>
<pre><code>22 13 17 <em>11</em>  0         3 15  0  2 22        14 21 17 24  <em>4</em>
 8  2 23  <em>4</em> 24         <em>9</em> 18 13 17  <em>5</em>        10 16 15  <em>9</em> 19
21  <em>9</em> 14 16  <em>7</em>        19  8  <em>7</em> 25 23        18  8 23 26 20
 6 10  3 18  <em>5</em>        20 <em>11</em> 10 24  <em>4</em>        22 <em>11</em> 13  6  <em>5</em>
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  <em>7</em>
</code></pre>
<p>After the next six numbers are drawn (<code>17</code>, <code>23</code>, <code>2</code>, <code>0</code>, <code>14</code>, and <code>21</code>), there are still no winners:</p>
<pre><code>22 13 <em>17</em> <em>11</em>  <em>0</em>         3 15  <em>0</em>  <em>2</em> 22        <em>14</em> <em>21</em> <em>17</em> 24  <em>4</em>
 8  <em>2</em> <em>23</em>  <em>4</em> 24         <em>9</em> 18 13 <em>17</em>  <em>5</em>        10 16 15  <em>9</em> 19
<em>21</em>  <em>9</em> <em>14</em> 16  <em>7</em>        19  8  <em>7</em> 25 <em>23</em>        18  8 <em>23</em> 26 20
 6 10  3 18  <em>5</em>        20 <em>11</em> 10 24  <em>4</em>        22 <em>11</em> 13  6  <em>5</em>
 1 12 20 15 19        <em>14</em> <em>21</em> 16 12  6         <em>2</em>  <em>0</em> 12  3  <em>7</em>
</code></pre>
<p>Finally, <code>24</code> is drawn:</p>
<pre><code>22 13 <em>17</em> <em>11</em>  <em>0</em>         3 15  <em>0</em>  <em>2</em> 22        <em>14</em> <em>21</em> <em>17</em> <em>24</em>  <em>4</em>
 8  <em>2</em> <em>23</em>  <em>4</em> <em>24</em>         <em>9</em> 18 13 <em>17</em>  <em>5</em>        10 16 15  <em>9</em> 19
<em>21</em>  <em>9</em> <em>14</em> 16  <em>7</em>        19  8  <em>7</em> 25 <em>23</em>        18  8 <em>23</em> 26 20
 6 10  3 18  <em>5</em>        20 <em>11</em> 10 <em>24</em>  <em>4</em>        22 <em>11</em> 13  6  <em>5</em>
 1 12 20 15 19        <em>14</em> <em>21</em> 16 12  6         <em>2</em>  <em>0</em> 12  3  <em>7</em>
</code></pre>
<p>At this point, the third board <em>wins</em> because it has at least one complete row or column of marked numbers (in this case, the entire top row is marked: <code><em>14 21 17 24  4</em></code>).</p>
<p>The <em>score</em> of the winning board can now be calculated. Start by finding the <em>sum of all unmarked numbers</em> on that board; in this case, the sum is <code>188</code>. Then, multiply that sum by <em>the number that was just called</em> when the board won, <code>24</code>, to get the final score, <code>188 * 24 = <em>4512</em></code>.</p>
<p>To guarantee victory against the giant squid, figure out which board will win first. <em>What will your final score be if you choose that board?</em></p>


## Solution

NumPy's masked arrays should streamline this.

In [2]:
class BingoSimulator:
    def __init__(self, draws: Iterable[int], boards: npt.NDArray[int]):
        self._latest_draw = None
        self._draws = iter(draws)
        try:
            self._boards = np.ma.array(
                data=boards,
                mask=np.zeros((len(boards), 5, 5), dtype=bool),
                dtype=int
            )
        except np.ma.MaskError:
            ValueError('One or more 5x5 boards of integers must be provided.')

    @property
    def latest_draw(self) -> int:
        return self._latest_draw

    @property
    def boards(self) -> npt.NDArray[int]:
        return self._boards

    @property
    def boards_containing_bingos(self) -> List[int]:
        return self.boards[
            self.boards.mask.all(axis=1).any(axis=1)
            | self.boards.mask.all(axis=2).any(axis=1)
        ]

    def draw_number(self) -> None:
        try:
            self._latest_draw = next(self._draws)
        except StopIteration:
            raise ValueError('No draws remaining')
        self.boards.mask[self.boards == self.latest_draw] = True

    @classmethod
    def parse_bingo_subsystem_output(cls, output: str) -> 'BingoGame':
        output_lines = [line for line in output.strip().split('\n') if line]
        return BingoSimulator(
            draws=[
                int(number)
                for number in output_lines[0].strip().split(',')
            ],
            boards=np.array([
                [int(number) for number in line.split()]
                for line in output_lines[1:]
            ]).reshape((-1, 5, 5))
        )

### Testing `BingoSimulator`

In [3]:
example_bingo_subsystem_output = '''
    7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

    22 13 17 11  0
     8  2 23  4 24
    21  9 14 16  7
     6 10  3 18  5
     1 12 20 15 19

     3 15  0  2 22
     9 18 13 17  5
    19  8  7 25 23
    20 11 10 24  4
    14 21 16 12  6

    14 21 17 24  4
    10 16 15  9 19
    18  8 23 26 20
    22 11 13  6  5
     2  0 12  3  7
'''
example_bingo_game = BingoSimulator.parse_bingo_subsystem_output(
    example_bingo_subsystem_output
)
example_draws = example_bingo_subsystem_output.strip().split('\n')[0].split(',')

expected_winning_board = np.array([[
    [14, 21, 17, 24,  4],
    [10, 16, 15,  9, 19],
    [18,  8, 23, 26, 20],
    [22, 11, 13,  6,  5],
    [ 2,  0, 12,  3,  7]
]])
expected_turn_of_first_bingo = 12

for _ in example_draws:
    example_bingo_game.draw_number()
    if example_bingo_game.boards_containing_bingos.size:
        break

assert example_bingo_game.latest_draw == 24
assert example_bingo_game.boards_containing_bingos[0].sum() == 188
np.testing.assert_array_equal(
    example_bingo_game.boards_containing_bingos.data,
    expected_winning_board
)

### Answer

In [4]:
simulated_bingo_game = BingoSimulator.parse_bingo_subsystem_output(
    PUZZLE_INPUT.read_text()
)

while not simulated_bingo_game.boards_containing_bingos.size:
    simulated_bingo_game.draw_number()

(
    simulated_bingo_game.boards_containing_bingos[0].sum()
    * simulated_bingo_game.latest_draw
)

5685

<h2 id="part2">--- Part Two ---</h2><p>On the other hand, it might be wise to try a different strategy: <span title="That's 'cuz a submarine don't pull things' antennas out of their sockets when they lose. Giant squid are known to do that.">let the giant squid win</span>.</p>
<p>You aren't sure how many bingo boards a giant squid could play at once, so rather than waste time counting its arms, the safe thing to do is to <em>figure out which board will win last</em> and choose that one. That way, no matter which boards it picks, it will win for sure.</p>
<p>In the above example, the second board is the last to win, which happens after <code>13</code> is eventually called and its middle column is completely marked. If you were to keep playing until this point, the second board would have a sum of unmarked numbers equal to <code>148</code> for a final score of <code>148 * 13 = <em>1924</em></code>.</p>
<p>Figure out which board will win last. <em>Once it wins, what would its final score be?</em></p>


### Solution
I got away with using a dirt simple check for bingos in Part 1 that, among other niceties, didn't involve tracking the order bingos appear across multiple boards.
We can get away with using the same simulator here without modification at the cost of efficiency.

In [ ]:
simulated_bingo_game = BingoSimulator.parse_bingo_subsystem_output(
    PUZZLE_INPUT.read_text()
)

while simulated_bingo_game.boards_containing_bingos.size:
    simulated_bingo_game.draw_number()

(
    simulated_bingo_game.boards_containing_bingos[0].sum()
    * simulated_bingo_game.latest_draw
)

In [ ]:
class BingoSimulator:
    def __init__(self, draws: Iterable[int], boards: npt.NDArray[int]):
        self._latest_draw = None
        self._draws = iter(draws)
        try:
            self._boards = np.ma.array(
                data=boards,
                mask=np.zeros((len(boards), 5, 5), dtype=bool),
                dtype=int
            )
        except np.ma.MaskError:
            ValueError('One or more 5x5 boards of integers must be provided.')

    @property
    def latest_draw(self) -> int:
        return self._latest_draw

    @property
    def boards(self) -> npt.NDArray[int]:
        return self._boards

    @property
    def boards_containing_bingos(self) -> List[int]:
        return self.boards[
            self.boards.mask.all(axis=1).any(axis=1)
            | self.boards.mask.all(axis=2).any(axis=1)
        ]

    def draw_number(self) -> None:
        try:
            self._latest_draw = next(self._draws)
        except StopIteration:
            raise ValueError('No draws remaining')
        self.boards.mask[self.boards == self.latest_draw] = True

    @classmethod
    def parse_bingo_subsystem_output(cls, output: str) -> 'BingoGame':
        output_lines = [line for line in output.strip().split('\n') if line]
        return BingoSimulator(
            draws=[
                int(number)
                for number in output_lines[0].strip().split(',')
            ],
            boards=np.array([
                [int(number) for number in line.split()]
                for line in output_lines[1:]
            ]).reshape((-1, 5, 5))
        )